Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/ml-frameworks/scikit-learn/train-hyperparameter-tune-deploy-with-sklearn/train-hyperparameter-tune-deploy-with-sklearn.png)

# Train and hyperparameter tune on Iris Dataset with Scikit-learn
In this tutorial, we demonstrate how to use the Azure ML Python SDK to train a support vector machine (SVM) on a single-node CPU with Scikit-learn to perform classification on the popular [Iris dataset](https://archive.ics.uci.edu/ml/datasets/iris). We will also demonstrate how to perform hyperparameter tuning of the model using Azure ML's HyperDrive service.

## Prerequisites

* Go through the [Configuration](../../../../configuration.ipynb) notebook to install the Azure Machine Learning Python SDK and create an Azure ML Workspace

In [1]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (pyarrow 7.0.0 (c:\users\janguy\anaconda3\envs\dlresearch\lib\site-packages), Requirement.parse('pyarrow<4.0.0,>=0.17.0'), {'azureml-dataset-runtime'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.PipelineRun = azureml.pipeline.core.run:PipelineRun._from_dto with exception (azureml-core 1.39.0 (c:\users\janguy\anaconda3\envs\dlresearch\lib\site-packages), Requirement.parse('azureml-core~=1.38.0')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.ReusedStepRun = azureml.pipeline.core.run:StepRun._from_reused_dto with exception (azureml-core 1.39.0 (c:\users\janguy\anaconda3\envs\dlresearch\lib\site-packages), Requirement.parse('azureml-core~=1.38.0')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.StepRun = azureml.pipeline.core

SDK version: 1.39.0


## Diagnostics

Opt-in diagnostics for better experience, quality, and security of future releases.

In [ ]:
from azureml.telemetry import set_diagnostics_collection

set_diagnostics_collection(send_diagnostics=True)

## Initialize workspace

Initialize a [Workspace](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#workspace) object from the existing workspace you created in the Prerequisites step. `Workspace.from_config()` creates a workspace object from the details stored in `config.json`.

In [1]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (pyarrow 7.0.0 (c:\users\janguy\anaconda3\envs\dlresearch\lib\site-packages), Requirement.parse('pyarrow<4.0.0,>=0.17.0'), {'azureml-dataset-runtime'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.PipelineRun = azureml.pipeline.core.run:PipelineRun._from_dto with exception (azureml-core 1.39.0 (c:\users\janguy\anaconda3\envs\dlresearch\lib\site-packages), Requirement.parse('azureml-core~=1.38.0')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.ReusedStepRun = azureml.pipeline.core.run:StepRun._from_reused_dto with exception (azureml-core 1.39.0 (c:\users\janguy\anaconda3\envs\dlresearch\lib\site-packages), Requirement.parse('azureml-core~=1.38.0')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.StepRun = azureml.pipeline.core

Workspace name: ws01ent
Azure region: westus2
Subscription id: 0e9bace8-7a81-4922-83b5-d995ff706507
Resource group: azureml


## Create AmlCompute

You will need to create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) for training your model. In this tutorial, we use Azure ML managed compute ([AmlCompute](https://docs.microsoft.com/azure/machine-learning/service/how-to-set-up-training-targets#amlcompute)) for our remote training compute resource.

> Note that if you have an AzureML Data Scientist role, you will not have permission to create compute resources. Talk to your workspace or IT admin to create the compute targets described in this section, if they do not already exist.

As with other Azure services, there are limits on certain resources (e.g. AmlCompute) associated with the Azure Machine Learning service. Please read [this article](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-manage-quotas) on the default limits and how to request more quota.

If we could not find the cluster with the given name, then we will create a new cluster here. We will create an `AmlCompute` cluster of `STANDARD_D2_V2` CPU VMs. This process is broken down into 3 steps:
1. create the configuration (this step is local and only takes a second)
2. create the cluster (this step will take about **20 seconds**)
3. provision the VMs to bring the cluster to the initial size (of 1 in this case). This step will take about **3-5 minutes** and is providing only sparse output in the process. Please make sure to wait until the call returns before moving to the next cell

In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "DS11"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', 
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

# can poll for a minimum number of nodes and for a specific timeout. 
# if no min node count is provided it uses the scale settings for the cluster
compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

Found existing compute target
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 2, 'targetNodeCount': 2, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 2, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2022-05-11T14:54:01.094000+00:00', 'errors': None, 'creationTime': '2022-03-02T18:08:36.882479+00:00', 'modifiedTime': '2022-03-18T19:29:05.264197+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 10, 'nodeIdleTimeBeforeScaleDown': 'PT1800S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_DS11_V2'}


The above code retrieves a CPU compute target. Scikit-learn does not support GPU computing.

## Train model on the remote compute

Now that you have your data and training script prepared, you are ready to train on your remote compute. You can take advantage of Azure compute to leverage a CPU cluster.

### Create a project directory

Create a directory that will contain all the necessary code from your local machine that you will need access to on the remote resource. This includes the training script and any additional files your training script depends on.

In [7]:
import os

project_folder = './sklearn-iris'
os.makedirs(project_folder, exist_ok=True)

### Prepare training script

Now you will need to create your training script. In this tutorial, the training script is already provided for you at `train_iris`.py. In practice, you should be able to take any custom training script as is and run it with Azure ML without having to modify your code.

However, if you would like to use Azure ML's [tracking and metrics](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#metrics) capabilities, you will have to add a small amount of Azure ML code inside your training script.

In `train_iris.py`, we will log some metrics to our Azure ML run. To do so, we will access the Azure ML Run object within the script:

```python
from azureml.core.run import Run
run = Run.get_context()
```

Further within `train_iris.py`, we log the kernel and penalty parameters, and the highest accuracy the model achieves:

```python
run.log('Kernel type', np.string(args.kernel))
run.log('Penalty', np.float(args.penalty))

run.log('Accuracy', np.float(accuracy))
```

These run metrics will become particularly important when we begin hyperparameter tuning our model in the "Tune model hyperparameters" section.

Once your script is ready, copy the training script `train_iris.py` into your project directory.

In [6]:
import shutil

shutil.copy('train_iris.py', project_folder)

'./sklearn-iris\\train_iris.py'

### Create an experiment

Create an [Experiment](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#experiment) to track all the runs in your workspace for this Scikit-learn tutorial.

In [3]:
from azureml.core import Experiment

experiment_name = 'train_iris'
experiment = Experiment(ws, name=experiment_name)

### Create an environment

Define a conda environment YAML file with your training script dependencies and create an Azure ML environment.

In [9]:
%%writefile sklearn-iris/conda_dependencies.yml

dependencies:
- python=3.8.5
- scikit-learn
- pip:
  - azureml-defaults

Overwriting sklearn-iris/conda_dependencies.yml


In [4]:
from azureml.core import Environment

sklearn_env = Environment.from_conda_specification(name = 'sklearn-env', file_path = 'sklearn-iris//conda_dependencies.yml')

### Configure the training job

Create a ScriptRunConfig object to specify the configuration details of your training job, including your training script, environment to use, and the compute target to run on.

In [40]:
from azureml.core import ScriptRunConfig

src = ScriptRunConfig(source_directory=project_folder,
                      script='train_iris.py',
                      arguments=['--kernel', 'linear', '--penalty', 1.0],
                      compute_target=compute_target,
                      environment=sklearn_env)

### Submit job

Run your experiment by submitting your ScriptRunConfig object. Note that this call is asynchronous.

In [12]:
run = experiment.submit(src)

## Monitor your run

You can monitor the progress of the run with a Jupyter widget. Like the run submission, the widget is asynchronous and provides live updates every 10-15 seconds until the job completes.

In [12]:
from azureml.widgets import RunDetails

RunDetails(run).show()

In [13]:
run.wait_for_completion(show_output=True)

RunId: train_iris_1652282118_972cfca3
Web View: https://ml.azure.com/runs/train_iris_1652282118_972cfca3?wsid=/subscriptions/0e9bace8-7a81-4922-83b5-d995ff706507/resourcegroups/azureml/workspaces/ws01ent&tid=72f988bf-86f1-41af-91ab-2d7cd011db47

Streaming azureml-logs/20_image_build_log.txt

2022/05/11 15:15:24 Downloading source code...
2022/05/11 15:15:24 Finished downloading source code
2022/05/11 15:15:25 Creating Docker network: acb_default_network, driver: 'bridge'
2022/05/11 15:15:25 Successfully set up Docker network: acb_default_network
2022/05/11 15:15:25 Setting up Docker configuration...
2022/05/11 15:15:26 Successfully set up Docker configuration
2022/05/11 15:15:26 Logging in to registry: ws01ent4ce6f8a0.azurecr.io
2022/05/11 15:15:26 Successfully logged into ws01ent4ce6f8a0.azurecr.io
2022/05/11 15:15:26 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2022/05/11 15:15:26 Scanning for dependencies...
2022/05/11 15:1

## Tune model hyperparameters

Now that we've seen how to do a simple Scikit-learn training run using the SDK, let's see if we can further improve the accuracy of our model. We can optimize our model's hyperparameters using Azure Machine Learning's hyperparameter tuning capabilities.

### Start a hyperparameter sweep

First, we will define the hyperparameter space to sweep over. Let's tune the `kernel` and `penalty` parameters. In this example we will use random sampling to try different configuration sets of hyperparameters to maximize our primary metric, `Accuracy`.

In [41]:
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.parameter_expressions import choice
    

param_sampling = RandomParameterSampling( {
    "--kernel": choice('linear', 'rbf'),
    "--penalty": choice(0.5, 1)
    }
)

hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=param_sampling, 
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=12,
                                     max_concurrent_runs=4)

Finally, lauch the hyperparameter tuning job.

In [42]:
# start the HyperDrive run
hyperdrive_run = experiment.submit(hyperdrive_config)

Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


## Monitor HyperDrive runs

You can monitor the progress of the runs with the following Jupyter widget.

In [43]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [44]:
hyperdrive_run.wait_for_completion(show_output=True)

RunId: HD_9c0b4787-5dca-451b-969e-7be2b0a60235
Web View: https://ml.azure.com/runs/HD_9c0b4787-5dca-451b-969e-7be2b0a60235?wsid=/subscriptions/0e9bace8-7a81-4922-83b5-d995ff706507/resourcegroups/azureml/workspaces/ws01ent&tid=72f988bf-86f1-41af-91ab-2d7cd011db47

Streaming azureml-logs/hyperdrive.txt

"<START>[2022-05-11T16:15:38.639647][API][INFO]Experiment created<END>\n""<START>[2022-05-11T16:15:39.356204][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n"<START>[2022-05-11T16:15:40.1248170Z][SCHEDULER][INFO]Scheduling job, id='HD_9c0b4787-5dca-451b-969e-7be2b0a60235_0'<END><START>[2022-05-11T16:15:40.2691215Z][SCHEDULER][INFO]Scheduling job, id='HD_9c0b4787-5dca-451b-969e-7be2b0a60235_1'<END><START>[2022-05-11T16:15:40.4569954Z][SCHEDULER][INFO]Scheduling job, id='HD_9c0b4787-5dca-451b-969e-7be2b0a60235_2'<END>"<START>[2022-05-11T16:15:40.461906][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<

Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms


Execution Summary
RunId: HD_9c0b4787-5dca-451b-969e-7be2b0a60235
Web View: https://ml.azure.com/runs/HD_9c0b4787-5dca-451b-969e-7be2b0a60235?wsid=/subscriptions/0e9bace8-7a81-4922-83b5-d995ff706507/resourcegroups/azureml/workspaces/ws01ent&tid=72f988bf-86f1-41af-91ab-2d7cd011db47



{'runId': 'HD_9c0b4787-5dca-451b-969e-7be2b0a60235',
 'target': 'DS11',
 'status': 'Completed',
 'startTimeUtc': '2022-05-11T16:15:38.381287Z',
 'endTimeUtc': '2022-05-11T16:21:11.567784Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '7eaa1a22-6c40-4fb9-8066-2cdc4009858b',
  'user_agent': 'python/3.8.5 (Windows-10-10.0.22000-SP0) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.39.0',
  'space_size': '4',
  'score': '0.9736842105263158',
  'best_child_run_id': 'HD_9c0b4787-5dca-451b-969e-7be2b0a60235_0',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://ws01ent3218162019.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_9c0b4787-5dca-451b-969e-7be2b0a60235/azureml-logs/hyperdrive.txt?sv=2019-07-

In [45]:
assert(hyperdrive_run.get_status() == "Completed")

### Warm start a Hyperparameter Tuning experiment and resuming child runs
Often times, finding the best hyperparameter values for your model can be an iterative process, needing multiple tuning runs that learn from previous hyperparameter tuning runs. Reusing knowledge from these previous runs will accelerate the hyperparameter tuning process, thereby reducing the cost of tuning the model and will potentially improve the primary metric of the resulting model. When warm starting a hyperparameter tuning experiment with Bayesian sampling, trials from the previous run will be used as prior knowledge to intelligently pick new samples, so as to improve the primary metric. Additionally, when using Random or Grid sampling, any early termination decisions will leverage metrics from the previous runs to determine poorly performing training runs. 

Azure Machine Learning allows you to warm start your hyperparameter tuning run by leveraging knowledge from up to 5 previously completed hyperparameter tuning parent runs. 

Additionally, there might be occasions when individual training runs of a hyperparameter tuning experiment are cancelled due to budget constraints or fail due to other reasons. It is now possible to resume such individual training runs from the last checkpoint (assuming your training script handles checkpoints). Resuming an individual training run will use the same hyperparameter configuration and mount the storage used for that run. The training script should accept the "--resume-from" argument, which contains the checkpoint or model files from which to resume the training run. You can also resume individual runs as part of an experiment that spends additional budget on hyperparameter tuning. Any additional budget, after resuming the specified training runs is used for exploring additional configurations.

For more information on warm starting and resuming hyperparameter tuning runs, please refer to the [Hyperparameter Tuning for Azure Machine Learning documentation](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-tune-hyperparameters) 

### Find and register best model
When all jobs finish, we can find out the one that has the highest accuracy.

In [46]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])

['--kernel', 'linear', '--penalty', '1', '--kernel', 'linear', '--penalty', '1']


Now, let's list the model files uploaded during the run.

In [47]:
print(best_run.get_file_names())

['outputs/model.joblib', 'system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'user_logs/std_log.txt']


We can then register the folder (and all files in it) as a model named `sklearn-iris` under the workspace for deployment

In [48]:
model = best_run.register_model(model_name='sklearn-iris', model_path='outputs/model.joblib')

In [50]:
model.download("",exist_ok =True)

'model.joblib'

In [51]:
import joblib
model = joblib.load("model.joblib")

Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


In [23]:
from sklearn import datasets
dataset = datasets.load_iris()
dataset.target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [24]:
dataset.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


In [52]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
dataset = pd.read_csv("https://azuremlexamples.blob.core.windows.net/datasets/iris.csv")
X= dataset[['sepal_length','sepal_width','petal_length','petal_width']]
predictions = model.predict(X)
labels = ['setosa', 'versicolor', 'virginica']
encoder = LabelEncoder()
encoder.fit(labels)


X has feature names, but SVC was fitted without feature names


LabelEncoder()

In [66]:
import datetime
df1 = pd.DataFrame()
probs = model.predict_proba(X)
probs = [prob for prob in probs]
df1['probs'] = probs
preds = model.predict(X)
df1['pred'] = preds
df1['scoring_service'] = "managed_online"
ts =datetime.datetime.now()
df1['timestamp'] = ts
df1

X has feature names, but SVC was fitted without feature names
X has feature names, but SVC was fitted without feature names


,probs,pred,scoring_service,timestamp
0,"[0.9687495487370514, 0.0200399419525324, 0.011...",0,managed_online,2022-05-11 09:27:43.879118
1,"[0.9518729166265446, 0.034112116228648695, 0.0...",0,managed_online,2022-05-11 09:27:43.879118
2,"[0.9668718378877207, 0.021691630111445353, 0.0...",0,managed_online,2022-05-11 09:27:43.879118
3,"[0.9471423884621806, 0.03677430221665482, 0.01...",0,managed_online,2022-05-11 09:27:43.879118
4,"[0.971344605070531, 0.01777855035556385, 0.010...",0,managed_online,2022-05-11 09:27:43.879118
...,...,...,...,...
145,"[0.013629398016075668, 0.04007509585001808, 0....",2,managed_online,2022-05-11 09:27:43.879118
146,"[0.016817428311098573, 0.1045201855985638, 0.8...",2,managed_online,2022-05-11 09:27:43.879118
147,"[0.015031306733820669, 0.09066008489871212, 0....",2,managed_online,2022-05-11 09:27:43.879118
148,"[0.011507042544094036, 0.011874800996062026, 0...",2,managed_online,2022-05-11 09:27:43.879118


Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
